# The Research Proposal

## The Problem

Historically in Major League Baseball (MLB) there have been a small set of statistics from gameplay that have been the considered the most important to determining if a team will have success (win) or not (lose). These stats are the pitcher's earned run average (ERA) and the batter's batting avereage. In more recent times there are a growing number of statisticians that are less convinced that these stats correlate directly to team success. We need to determine if the higher a team's batting average, and the lower it's ERA, correlate significantly to that team's win-loss record. For now I will focus on batting average.

## The Potential Solution - Hypothesis

Hyothesis: Teams with higher batting averages have more success.

Null: There is no difference in success for teams with higher batting averages.

## The Method of Testing the Solution

Will be conducting A-B testing to determine if higher batting averages are better at predicting success then teams without. The sample I'll be using is all MLB win/loss records for every team since 1950 as well as the batting averages for a given team during that timeframe. I will consider a confidence interval of 95% or greater a success.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

First I'll import the CSV file with teams, wins and losses. 

Next I'll import the batting stats for every player.

In [28]:
teams = pd.read_csv('Teams.csv', index_col=0)

In [29]:
batting = pd.read_csv('Batting.csv', index_col=0)

In [30]:
teams.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
yearTeamID,,,,,,,,,,,,,,,,,,,,,
1871BS1,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1871CH1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
1871CL1,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
1871FW1,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
1871NY2,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2


In [31]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
yearTeamID,,,,,,,,,,,,,,,,,,,,,
1871TRO,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1871RC1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
1871CL1,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
1871WS3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
1871RC1,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


Lets keep it to modern teams. After WW2 with a 5 year buffer should work.

In [32]:
teams.drop(teams[(teams.yearID < 1950)].index, inplace=True)

In [33]:
batting.drop(batting[(batting.yearID < 1950)].index, inplace=True)

In [34]:
teams.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
yearTeamID,,,,,,,,,,,,,,,,,,,,,
1950BOS,1950,AL,BOS,BOS,NaN,3,154,77.0,94,60,...,181,0.981,Boston Red Sox,Fenway Park II,1344080.0,111,108,BOS,BOS,BOS
1950BRO,1950,NL,BRO,LAD,NaN,2,155,78.0,89,65,...,183,0.979,Brooklyn Dodgers,Ebbets Field,1185896.0,103,99,BRO,BRO,BRO
1950BSN,1950,NL,BSN,ATL,NaN,4,156,79.0,83,71,...,146,0.970,Boston Braves,Braves Field,944391.0,94,93,BSN,BSN,BSN
1950CHA,1950,AL,CHA,CHW,NaN,6,156,79.0,60,94,...,181,0.977,Chicago White Sox,Comiskey Park,781330.0,97,98,CHW,CHA,CHA
1950CHN,1950,NL,CHN,CHC,NaN,7,154,78.0,64,89,...,169,0.968,Chicago Cubs,Wrigley Field,1165944.0,99,102,CHC,CHN,CHN


In [35]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
yearTeamID,,,,,,,,,,,,,,,,,,,,,
1950CLE,aberal01,1950,1,CLE,AL,1,2,0,0,0,...,0.0,0.0,0.0,1,1.0,NaN,0.0,0.0,NaN,0.0
1950BRO,abramca01,1950,1,BRO,NL,38,44,5,9,1,...,4.0,0.0,NaN,9,13.0,NaN,0.0,1.0,NaN,0.0
1950CIN,adamsbo03,1950,1,CIN,NL,115,348,57,98,21,...,25.0,7.0,NaN,43,29.0,NaN,0.0,3.0,NaN,2.0
1950CHA,adamshe01,1950,1,CHA,AL,34,118,12,24,2,...,2.0,3.0,0.0,12,7.0,NaN,2.0,1.0,NaN,2.0
1950CIN,adcocjo01,1950,1,CIN,NL,102,372,46,109,16,...,55.0,2.0,NaN,24,24.0,NaN,0.0,3.0,NaN,12.0


Now I need to group batting by team and year, luckily I already have the yearTeamID to which to do this with.

In [ ]:
bat